# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.Timestepping;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ],,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\smuda\binaries,False,FDY-WindowsHPC,dotnet,,[ \\hpccluster\hpccluster-scratch\smuda\ == ],FDY\smuda,DC2,<null>,Normal,True


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [ ]:
//var myBatch = ExecutionQueues[0];
var myBatch = GetDefaultQueue();
myBatch

DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths
D:\local\binaries,False,LocalPC,dotnet,<null>,[ D:\local\ == ]


A switch for running the worksheet locally (since we have a folder structure for the data, which is not copied on Jenkins for testing)

In [ ]:
bool NunitTest = false;

In [ ]:
//foreach(var s in wmg.Sessions)
//    s.Delete(true);

## Verification of Initial Value data

Initial values and parameters for the simulation (intial droplet shape, Ohnesorg number, initial velocity)
were specified by project partner (TU Graz, Group Prof. Brenn).
Details can be found in Document `setup.pdf`, to be found in the same directory as this worksheet.

First, it is verified that the initial values chosen here actually match the specification.

In [ ]:
//MultidimensionalArray[] ReferenceData = new MultidimensionalArray[5];
int[] modes = new int[] { 2, 3, 4 };
List<string> cases = new List<string>();
int nCase = 0;
// add simulations for mode 2
if(modes.Contains(2)) {
    cases.Add("m2_Oh01_eta04");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta02");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m2_Oh01_eta01");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
// add simulations for mode 3
if(modes.Contains(3)) {
    cases.Add("m3_Oh01_eta04");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m3_Oh01_eta015");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
// add simulations for mode 4
if(modes.Contains(4)) {
    cases.Add("m4_Oh01_eta04");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh01_eta01");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
    cases.Add("m4_Oh056_eta005");
    Console.WriteLine($"case {nCase+1}: {cases[nCase]}");
    nCase++;
}
int numCases = cases.Count;
numCases

8

Load reference data for all cases; These files contain two columns, i.e. the azimuth angle and the respective droplet radius.

In [ ]:
MultidimensionalArray[] ReferenceData = new MultidimensionalArray[numCases];
for(int iCase = 0; iCase < numCases; iCase++) {
    if(!NunitTest)  
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/surfaceDrop_{cases[iCase]}.txt");
    else
        ReferenceData[iCase] = IMatrixExtensions.LoadFromTextFile($"surfaceDrop_{cases[iCase]}.txt");
}

Analytical expressions for the reference data (see `setup.pdf`); this is to verify that the definition of Legendre
Functions resp. Polynomials in BoSSS actually matches the definition used by the TU Graz group.

In [ ]:
Dictionary<string, Func<double, double>> casesRadius = new Dictionary<string, Func<double, double>>();

In [ ]:
double caseR_m2eta04(double angle) {
   double radius = 0.966781 + 0.4*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta04", caseR_m2eta04);

In [ ]:
double caseR_m2eta02(double angle) {
   double radius = 0.991848 + 0.2*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta02", caseR_m2eta02);

In [ ]:
double caseR_m2eta01(double angle) {
   double radius = 0.997981 + 0.1*SphericalHarmonics.MyLegendre(2,0,Math.Cos(angle));
   return radius; 
}
casesRadius.Add("m2_Oh01_eta01", caseR_m2eta01);

In [ ]:
double caseR_m3eta04(double angle) {
    double radius = 0.977143 + 0.4*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta04", caseR_m3eta04);

In [ ]:
double caseR_m3eta015(double angle) {
    double radius = 0.996786 + 0.15*SphericalHarmonics.MyLegendre(3,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m3_Oh01_eta015", caseR_m3eta015);

In [ ]:
double caseR_m4eta04(double angle) {
    double radius = 0.981839 + 0.4*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta04", caseR_m4eta04);

In [ ]:
double caseR_m4eta01(double angle) {
    double radius = 0.998883 + 0.1*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh01_eta01", caseR_m4eta01);

In [ ]:
double caseR_m4eta005(double angle) {
    double radius = 0.999721 + 0.05*SphericalHarmonics.MyLegendre(4,0,Math.Cos(angle));
    return radius; 
}
casesRadius.Add("m4_Oh056_eta005", caseR_m4eta005);

Conversion to cartesian coordinates in order to match the data and verification against analytical expression:

In [ ]:
double[][] refX = new double[numCases][];
double[][] refZ = new double[numCases][];
double[][] caseX = new double[numCases][];
double[][] caseZ = new double[numCases][];
for(int iCase = 0; iCase < numCases; iCase++) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    double[] radius = ReferenceData[iCase].GetColumn(1);
       
    double RadiusErrorNorm = 0.0;
    int I = angle.Length;
    double[] x1 = new double[I], z1 = new double[I];
    double[] cx1 = new double[I], cz1 = new double[I];
    for(int i = 0; i < I; i++) {
        x1[i] = Math.Sin(angle[i])*radius[i];
        z1[i] = Math.Cos(angle[i])*radius[i];
        
        double radius_expr = casesRadius[cases[iCase]](angle[i]);
        RadiusErrorNorm += (radius[i] - radius_expr).Pow2();     
        
        cx1[i] = Math.Sin(angle[i])*radius_expr;
        cz1[i] = Math.Cos(angle[i])*radius_expr;
    } 
    refX[iCase] = x1;
    refZ[iCase] = z1;
    caseX[iCase] = cx1;
    caseZ[iCase] = cz1;

    RadiusErrorNorm = RadiusErrorNorm.Sqrt();
    Console.WriteLine($"Comparison error for radius in case {iCase + 1}: {RadiusErrorNorm}");
    // Note: since the factors in `setup.pdf` are only provided up to 6 digits, an error threshold of 1e-5 seems reasonable.
    Assert.LessOrEqual(RadiusErrorNorm, 1e-5, "Error in comparing reference data against Legendre polynomials in BoSSS");
}

Error: NUnit.Framework.AssertionException:   Error in comparing reference data against Legendre polynomials in BoSSS
  Expected: less than or equal to 1.0000000000000001E-05d
  But was:  0.0039783556464071616d

   at NUnit.Framework.Assert.ReportFailure(String message) in /_/src/NUnitFramework/framework/Assert.cs:line 395
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.cs:line 383
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.That.cs:line 229
   at NUnit.Framework.Assert.LessOrEqual(Double arg1, Double arg2, String message, Object[] args) in /_/src/NUnitFramework/framework/Assert.Comparisons.cs:line 943
   at Submission#26.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmis

### Plot of Reference Data

In [ ]:
int refCase = 4;
Plot(refX[refCase], refZ[refCase], "Ref-Case", ". black",
    caseX[refCase], caseZ[refCase], "Case", ". blue")

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1 
 
 
 
 
 -0.8 
 
 
 
 
 -0.6 
 
 
 
 
 -0.4 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 
 
 
 
 Ref-Case 
 
 
 Ref-Case 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M716.2,34.7 L758.4,34.7 M53.9,30.0 L60.8,30.0 L67.7,30.1 L74.6,30.2 L81.4,30.4 L88.3,30.7
 L95.2,30.9 L102.0,31.3 L108.8,31.6 L115.6,32.1 L122.4,32.5 L129.2,33.1 L135.9,33.7 L142.6,34.3
 L149.3,34.9 L156.0,35.7 L162.6,36.4 L169.2,37.3 L175.7,38.1 L182.2,39.0 L188.7,40.0 L195.1,41.0
 L201.5,42.0 L207.8,43.1 L214.1,44.2 L220.3,45.4 L226.5,46.6 L232.7,47.8 L238.8,49.1 L244.8,50.4
 L250.8,51.8 L256.7,53.2 L262.5,54.6 L268.3,56.1 L274.1,57.6 L279.7,59.1 L285.3,60.7 L290.9,62.3
 L296.4,63.9 L301.8,65.6 L307.2,67.2 L312.5,69.0 L317.7,70.7 L322.8,72.5 L327.9,74.2 L333.0,76.1
 L337.9,77.9 L342.8,79.8 L347.7,81.6 L352.4,83.5 L357.1,85.4 L361.8,87.4 L366.3,89.3 L370.8,91.3
 L375.3,93.3 L379.6,95.3 L383.9,97.3 L388.2,99.3 L392.4,101.4 L396.5,103.4 L400.5,105.5 L404.5,107.5
 L408.5,109.6 L412.4,111.7 L416.2,113.8 L420.0,115.9 L423.7,118.0 L427.3,120.1 L431.0,122.2 L434.5,124.3
 L438.0,126.4 L441.5,128.6 L444.9,130.7 L448.2,132.8 L451.6,134.9 L454.8,137.1 L458.1,139.2 L461.3,141.3
 L464.4,143.4 L467.5,145.6 L470.6,147.7 L473.6,149.8 L476.6,151.9 L479.6,154.1 L482.5,156.2 L485.4,158.3
 L488.3,160.4 L491.2,162.5 L494.0,164.6 L496.8,166.7 L499.6,168.8 L502.3,170.9 L505.1,173.0 L507.8,175.1
 L510.5,177.2 L513.1,179.3 L515.8,181.4 L518.4,183.4 L521.1,185.5 L523.7,187.6 L526.3,189.7 L528.9,191.7
 L531.4,193.8 L534.0,195.9 L536.6,197.9 L539.1,200.0 L541.7,202.1 L544.2,204.1 L546.7,206.2 L549.2,208.3
 L551.7,210.3 L554.2,212.4 L556.7,214.5 L559.2,216.5 L561.7,218.6 L564.2,220.7 L566.6,222.8 L569.1,224.9
 L571.6,226.9 L574.0,229.0 L576.4,231.1 L578.9,233.2 L581.3,235.3 L583.7,237.4 L586.1,239.6 L588.5,241.7
 L590.9,243.8 L593.3,246.0 L595.6,248.1 L598.0,250.3 L600.3,252.4 L602.6,254.6 L604.9,256.8 L607.2,259.0
 L609.5,261.2 L611.7,263.4 L613.9,265.6 L616.1,267.9 L618.3,270.1 L620.5,272.4 L622.6,274.6 L624.7,276.9
 L626.7,279.2 L628.8,281.5 L630.8,283.8 L632.7,286.1 L634.6,288.5 L636.5,290.8 L638.4,293.2 L640.2,295.6
 L641.9,298.0 L643.6,300.4 L645.3,302.8 L646.9,305.2 L648.4,307.6 L649.9,310.1 L651.4,312.6 L652.8,315.0
 L654.1,317.5 L655.4,320.0 L656.6,322.5 L657.7,325.0 L658.8,327.6 L659.8,330.1 L660.7,332.6 L661.6,335.2
 L662.4,337.8 L663.1,340.3 L663.7,342.9 L664.2,345.5 L664.7,348.1 L665.1,350.7 L665.4,353.3 L665.6,355.9
 L665.7,358.5 L665.8,361.1 L665.7,363.7 L665.6,366.4 L665.4,369.0 L665.0,371.6 L664.6,374.2 L664.1,376.8
 L663.5,379.4 L662.8,382.1 L661.9,384.7 L661.0,387.3 L660.0,389.9 L658.9,392.5 L657.7,395.1 L656.4,397.7
 L654.9,400.2 L653.4,402.8 L651.8,405.4 L650.0,407.9 L648.2,410.4 L646.3,413.0 L644.2,415.5 L642.1,417.9
 L639.8,420.4 L637.5,422.9 L635.0,425.3 L632.5,427.8 L629.8,430.2 L627.0,432.5 L624.2,434.9 L621.2,437.3
 L618.1,439.6 L615.0,441.9 L611.7,444.2 L608.4,446.4 L604.9,448.6 L601.4,450.8 L597.7,453.0 L594.0,455.2
 L590.2,457.3 L586.3,459.4 L582.3,461.4 L578.2,463.5 L574.0,465.5 L569.8,467.4 L565.5,469.4 L561.1,471.3
 L556.6,473.2 L552.0,475.0 L547.4,476.8 L542.7,478.6 L537.9,480.3 L533.1,482.0 L528.1,483.7 L523.2,485.3
 L518.1,486.9 L513.0,488.5 L507.9,490.0 L502.7,491.5 L497.4,493.0 L492.1,494.4 L486.8,495.8 L481.4,497.1
 L475.9,498.5 L470.4,499.7 L464.9,501.0 L459.3,502.2 L453.7,503.4 L448.1,504.5 L442.4,505.6 L436.8,506.6
 L431.0,507.7 L425.3,508.7 L419.5,509.6 L413.7,510.5 L407.9,511.4 L402.1,512.3 L396.3,513.1 L390.5,513.9
 L384.6,514.7 L378.7,515.4 L372.9,516.1 L367.0,516.8 L361.1,517.4 L355.3,518.0 L349.4,518.6 L343.5,5

### Matching of the Spherical Harmonics against the provided Data

In [ ]:
Dictionary<string, Formula> casesPhi = new Dictionary<string, Formula>();

In [ ]:
var Phi_m2eta04_Init = new Formula(
"Phi1",
false,
"using ilPSP.Utils; " + 
"double Phi1(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.966781*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"}");
casesPhi.Add("m2_Oh01_eta04", Phi_m2eta04_Init);

In [ ]:
var Phi_m2eta02_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.991848*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.2*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta02", Phi_m2eta02_Init);

In [ ]:
var Phi_m2eta01_Init = new Formula(
"Phi4",
false,
"using ilPSP.Utils; " + 
"double Phi4(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.997981*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.1*SphericalHarmonics.MyRealSpherical(2, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m2_Oh01_eta01", Phi_m2eta01_Init);

In [ ]:
var Phi_m3eta04_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.977143*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta04", Phi_m3eta04_Init);

In [ ]:
var Phi_m3eta015_Init = new Formula(
"Phi2",
false,
"using ilPSP.Utils; " + 
"double Phi2(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.996786*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.15*SphericalHarmonics.MyRealSpherical(3, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m3_Oh01_eta015", Phi_m3eta015_Init);

In [ ]:
var Phi_m4eta04_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.981839*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.4*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta04", Phi_m4eta04_Init);

In [ ]:
var Phi_m4eta01_Init = new Formula(
"Phi3",
false,
"using ilPSP.Utils; " + 
"double Phi3(double[] X) { " +    
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.998883*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.1*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh01_eta01", Phi_m4eta01_Init);

In [ ]:
var Phi_m4eta005_Init = new Formula(
"Phi5",
false,
"using ilPSP.Utils; " + 
"double Phi5(double[] X) { " + 
"    (double theta, double phi) = SphericalHarmonics.GetAngular(X); " + 
"    double R =    0.999721*SphericalHarmonics.MyRealSpherical(0, 0, theta, phi) " + 
"                +      0.05*SphericalHarmonics.MyRealSpherical(4, 0, theta, phi); " + 
"    return X.L2Norm() - R; " + 
"} ");
casesPhi.Add("m4_Oh056_eta005", Phi_m4eta005_Init);

In [ ]:
// IBoundaryAndInitialData[] Phi_iCase = new IBoundaryAndInitialData[]  { Phi1Init, Phi2Init, Phi3Init, Phi4Init, Phi5Init};

In [ ]:
for(int iCase = 0; iCase < numCases; iCase++) {
    double[] angle = ReferenceData[iCase].GetColumn(0);
    //double[] xI = refX[iCase];   
    //double[] zI = refZ[iCase];
    int I = angle.Length;
    
    double PhiErr = 0;
    for(int i = 0; i < I; i++) {
        double radius_expr = casesRadius[cases[iCase]](angle[i]);    
        double x1 = Math.Sin(angle[i])*radius_expr;
        double z1 = Math.Cos(angle[i])*radius_expr;
    
        PhiErr += casesPhi[cases[iCase]].Evaluate(new Vector(x1, 0, z1), 0.0).Abs();
    }
    Console.WriteLine($"Phi error for case {iCase}: {PhiErr}");
    Assert.LessOrEqual(PhiErr, 1e-10, "Level-Set function is not zero at desired surface.");
    
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e-5, 1e-5, 1e-5), 0.0) < 0, "Inside must be phase A/negative");
    Assert.IsTrue(casesPhi[cases[iCase]].Evaluate(new Vector(1e+1, 1e+1, 1e+1), 0.0) > 0, "Outside must be phase B/positive");
}

Phi error for case 0: 2.0650148258027912E-14
Phi error for case 1: 1.532107773982716E-14
Phi error for case 2: 1.0880185641326534E-14
Phi error for case 3: 2.2315482794965646E-14
Phi error for case 4: 3.530509218307998E-14
Phi error for case 5: 1.454392162258955E-14
Phi error for case 6: 1.2545520178264269E-14


### Initial Velocities

In [ ]:
//var anaVel = new IBoundaryAndInitialData[numCases,3];
Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)> analyticVel = new Dictionary<string, (IBoundaryAndInitialData velX, IBoundaryAndInitialData velY, IBoundaryAndInitialData velZ)>();

for(int iCase = 0; iCase < numCases; iCase++) {
    MultidimensionalArray polVel;
    MultidimensionalArray radVel;
    if (!NunitTest) {
        polVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/polarVel_{cases[iCase]}.txt");
        radVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/{cases[iCase].Substring(0,2)}/radialVel_{cases[iCase]}.txt");
    } else {
        polVel = IMatrixExtensions.LoadFromTextFile($"polarVel_{cases[iCase]}.txt");
        radVel = IMatrixExtensions.LoadFromTextFile($"radialVel_{cases[iCase]}.txt");
    }
    Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(0), radVel.GetColumn(0)));
    Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(1), radVel.GetColumn(1)));
    
    double[] radiusS = polVel.GetColumn(0);
    double[] anglesS = polVel.GetColumn(1);
    double[] polVelS = polVel.GetColumn(2);
    double[] radVelS = radVel.GetColumn(2);
    
    var velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
    velX.SetData(anglesS, radiusS, polVelS, radVelS);
    var velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
    velY.SetData(anglesS, radiusS, polVelS, radVelS);
    var velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
    velZ.SetData(anglesS, radiusS, polVelS, radVelS);
    
    // anaVel[iCase, 0] = velX;
    // anaVel[iCase, 1] = velY;
    // anaVel[iCase, 2] = velZ;
    analyticVel.Add(cases[iCase], (velX, velY, velZ));
}

In [ ]:
// var anaVel = new IBoundaryAndInitialData[3,3];

// MultidimensionalArray polVel;
// MultidimensionalArray radVel;
// if (!NunitTest) {
//     polVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/polarVelCase1.txt");
//     radVel = IMatrixExtensions.LoadFromTextFile($"data/InitialValues/radialVelCase1.txt");
// } else {
//     polVel = IMatrixExtensions.LoadFromTextFile($"polarVelCase1.txt");
//     radVel = IMatrixExtensions.LoadFromTextFile($"radialVelCase1.txt");
// }
// Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(0), radVel.GetColumn(0)));
// Assert.IsTrue(ilPSP.Utils.ArrayTools.ListEquals(polVel.GetColumn(1), radVel.GetColumn(1)));

// double[] radiusS = polVel.GetColumn(0);
// double[] anglesS = polVel.GetColumn(1);
// double[] polVelS = polVel.GetColumn(2);
// double[] radVelS = radVel.GetColumn(2);

// polVelS.ScaleV(-1.0);
// var velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
// velX.SetData(anglesS, radiusS, polVelS, radVelS);
// var velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
// velY.SetData(anglesS, radiusS, polVelS, radVelS);
// var velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
// velZ.SetData(anglesS, radiusS, polVelS, radVelS);

// anaVel[0, 0] = velX;
// anaVel[0, 1] = velY;
// anaVel[0, 2] = velZ;
// polVelS = polVel.GetColumn(2);

// radVelS.ScaleV(-1.0);
// velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
// velX.SetData(anglesS, radiusS, polVelS, radVelS);
// velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
// velY.SetData(anglesS, radiusS, polVelS, radVelS);
// velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
// velZ.SetData(anglesS, radiusS, polVelS, radVelS);

// anaVel[1, 0] = velX;
// anaVel[1, 1] = velY;
// anaVel[1, 2] = velZ;

// polVelS.ScaleV(-1.0);
// velX = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 0 };
// velX.SetData(anglesS, radiusS, polVelS, radVelS);
// velY = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 1 };
// velY.SetData(anglesS, radiusS, polVelS, radVelS);
// velZ = new BoSSS.Application.XNSE_Solver.SpecificSolutions.PolarAxiallySymmetricInitialValues() { VelocityComponent = 2 };
// velZ.SetData(anglesS, radiusS, polVelS, radVelS);

// anaVel[2, 0] = velX;
// anaVel[2, 1] = velY;
// anaVel[2, 2] = velZ;


## Grid Creation

In [ ]:
//foreach(var g in BoSSSshell.WorkflowMgm.Grids)
//   g.Delete(true);

### Quater-Domain grids
(Symmetry planes at $x = 0$ and $y = 0$)

In [ ]:
Dictionary<string, IGridInfo[]> gridTypes = new Dictionary<string, IGridInfo[]>();

In [ ]:
int[] Resolutions = new int[] { 6 };
IGridInfo[] Grids = new IGridInfo[Resolutions.Length];
double scale = 1.0;
for(int i = 0; i < Resolutions.Length; i++) {
    int Res = Resolutions[i];
    string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_wallBC_quarterDomain";

    IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
    //cachedGrid = null;
    if(cachedGrid == null) {
        
        // must create new Grid
        double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
        double[] yNodes = xNodes;
        double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
        var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        grd.Name = GridName;
        
        grd.DefineEdgeTags(delegate(Vector X) {
            string ret = null;
            if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
                ret = IncompressibleBcType.SlipSymmetry.ToString();
            else
                ret = IncompressibleBcType.Wall.ToString();
            return ret;
        });        
        
        Grids[i] = wmg.SaveGrid(grd);
        
    } else {
        //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
        Console.WriteLine("Grid already found in database - identifid by name " + GridName);
        Grids[i] = cachedGrid;
    }
    
}
gridTypes.Add("wallBC", Grids);

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.
Grid Edge Tags changed.
An equivalent grid (eb0268bd-e30e-4ffd-8b96-9f718b65012a) is already present in the database -- the grid will not be saved.


In [ ]:
// Resolutions = new int[] { 6 };
// Grids = new IGridInfo[Resolutions.Length];
// scale = 1.0;
// for(int i = 0; i < Resolutions.Length; i++) {
//     int Res = Resolutions[i];
//     string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_pressureOutletBC_quarterDomain";

//     IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
//     //cachedGrid = null;
//     if(cachedGrid == null) {
        
//         // must create new Grid
//         double[] xNodes = GenericBlas.Linspace(0, 3*scale, Res + 1);
//         double[] yNodes = xNodes;
//         double[] zNodes = GenericBlas.Linspace(-3*scale, 3*scale, Res*2 + 1);
        
//         var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
//         grd.Name = GridName;
        
//         grd.DefineEdgeTags(delegate(Vector X) {
//             string ret = null;
//             if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
//                 ret = IncompressibleBcType.SlipSymmetry.ToString();
//             else
//                 ret = IncompressibleBcType.Pressure_Outlet.ToString();
//             return ret;
//         });        
        
//         Grids[i] = wmg.SaveGrid(grd);
        
//     } else {
//         //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
//         Console.WriteLine("Grid already found in database - identifid by name " + GridName);
//         Grids[i] = cachedGrid;
//     }
    
// }
// gridTypes.Add("pressureOutletBC", Grids);

In [ ]:
// Resolutions = new int[] { 8 };
// Grids = new IGridInfo[Resolutions.Length];
// scale = 1.0;
// for(int i = 0; i < Resolutions.Length; i++) {
//     int Res = Resolutions[i];
//     string GridName = $"OscillatingDroplet3D_{Res}x{Res}x{2*Res}_pressureOutletBC_quarterDomain";

//     IGridInfo cachedGrid = wmg.Grids.FirstOrDefault(grid => grid.Name == GridName);
//     //cachedGrid = null;
//     if(cachedGrid == null) {
        
//         // must create new Grid
//         double[] xNodes = GenericBlas.Linspace(0, 2*scale, Res + 1);
//         double[] yNodes = xNodes;
//         double[] zNodes = GenericBlas.Linspace(-2*scale, 2*scale, Res*2 + 1);
        
//         var grd = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
//         grd.Name = GridName;
        
//         grd.DefineEdgeTags(delegate(Vector X) {
//             string ret = null;
//             if(X.x.Abs() <= 1e-8 || X.y.Abs() <= 1.0e-8)
//                 ret = IncompressibleBcType.SlipSymmetry.ToString();
//             else
//                 ret = IncompressibleBcType.Pressure_Outlet.ToString();
//             return ret;
//         });        
        
//         Grids[i] = wmg.SaveGrid(grd);
        
//     } else {
//         //Console.WriteLine($"type: {cachedGrid.GetType()}, is IGridInfo? {cachedGrid is IGridInfo}");
//         Console.WriteLine("Grid already found in database - identifid by name " + GridName);
//         Grids[i] = cachedGrid;
//     }
    
// }
// gridTypes.Add("refinedTest", Grids);

Grid already found in database - identifid by name OscillatingDroplet3D_8x8x16_pressureOutletBC_quarterDomain


In [ ]:
gridTypes

key,value
wallBC,[ { Guid = eb0268bd-e30e-4ffd-8b96-9f718b65012a; Name = OscillatingDroplet3D_6x6x12_quarterDomain; Cell Count = 432; Dim = 3 } ]


In [ ]:
//var g = (wmg.Grids[0] as GridProxy).RealGrid;

In [ ]:
//(g.iGridData as GridData).GlobalBoundingBox

In [ ]:
//wmg.Sessions[0].Delete(true);

## Setup of control objects for all solver runs

In [ ]:
cases

index,value
0,m2_Oh01_eta04
1,m2_Oh01_eta02
2,m2_Oh01_eta01
3,m3_Oh01_eta04
4,m4_Oh01_eta04
5,m4_Oh01_eta01
6,m4_Oh056_eta005


In [ ]:
Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)> casesSetUp = new Dictionary<string, (double Ohnesorge, int AMRlevel, double dt, int timesteps)>();

In [ ]:
casesSetUp.Add("m2_Oh01_eta04", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m2_Oh01_eta02", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m2_Oh01_eta01", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m3_Oh01_eta04", (0.1, 2, 2e-3, 3500));
casesSetUp.Add("m3_Oh01_eta015", (0.1, 2, 2e-3, 3500));
casesSetUp.Add("m4_Oh01_eta04", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m4_Oh01_eta01", (0.1, 1, 5e-3, 1400));
casesSetUp.Add("m4_Oh056_eta005", (0.56, 1, 5e-3, 800));

In [ ]:
string[] gridTypeKeys = new string[] { "wallBC" }; ///{ "wallBC", "pressureOutletBC", "refinedTest" };

In [ ]:
List<XNSE_Control> Controls = new List<XNSE_Control>();
Controls.Clear();
int[] DegreeS = new int[] { 3 };
bool[] useInitial = new bool[] { true, false };
bool[] useNewton = new bool[] { false };

// string grdKey = gridTypeKeys[1];
// Grids = gridTypes[grdKey];

foreach(bool bInitial in useInitial) {
foreach(bool bNewton in useNewton) {
foreach(int k in DegreeS) {
foreach(string grdKey in gridTypeKeys) {
foreach(var grd in gridTypes[grdKey]) {
int iCase = 0;
foreach(var myCase in cases) {
    long J = grd.NumberOfCells;
    int AMRlvl = casesSetUp[myCase].AMRlevel;
    string JobName = $"OD3D_J{J}k{k}_{grdKey}_amr{AMRlvl}_{myCase}";
    if(bInitial) {
        JobName = JobName + "_thirdOrderInit";
    }
    if(bNewton) {
        JobName = JobName + "_Newton";
    }
    Console.WriteLine($"Case {iCase+1}: " + JobName);
    iCase++;

    var C = new XNSE_Control();
    
    C.SetGrid(grd);
    C.SetDGdegree(k);
    C.SessionName = JobName;
    
    C.InitialValues.Add("Phi", casesPhi[myCase]);
    
    C.PhysicalParameters.IncludeConvection = true;
    C.PhysicalParameters.rho_A = 1;
    C.PhysicalParameters.rho_B = 0.001;
    C.PhysicalParameters.mu_A = casesSetUp[myCase].Ohnesorge;
    C.PhysicalParameters.mu_B = casesSetUp[myCase].Ohnesorge/1000;
    C.PhysicalParameters.reynolds_B = 0.0;
    C.PhysicalParameters.reynolds_A = 0.0;
    C.PhysicalParameters.Sigma = 1;
    C.PhysicalParameters.pFree = 0.0;
    C.PhysicalParameters.mu_I = 0.0;
    C.PhysicalParameters.lambda_I = 0.0;
    C.PhysicalParameters.lambdaI_tilde = -1.0;
    C.PhysicalParameters.betaS_A = 0.0;
    C.PhysicalParameters.betaS_B = 0.0;
    C.PhysicalParameters.betaL = 0.0;
    C.PhysicalParameters.theta_e = 1.5707963267948966;
    C.PhysicalParameters.sliplength = 0.0;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.useArtificialSurfaceForce = false;
    
    C.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.StokesExtension;
    C.AdvancedDiscretizationOptions.SST_isotropicMode = SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
    C.LSContiProjectionMethod = ContinuityProjectionOption.ConstrainedDG;
    
    C.TimeSteppingScheme = TimeSteppingScheme.BDF3;
    if(bNewton) {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    } else {
        C.NonLinearSolver.SolverCode = NonLinearSolverCode.Picard;
    }
    C.NonLinearSolver.ConvergenceCriterion = 1e-9;
    C.NonLinearSolver.MinSolverIterations = 3;
    C.Timestepper_BDFinit = TimeStepperInit.SingleInit;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.TimesteppingMode = AppControl._TimesteppingMode.Transient;
    C.dtFixed = casesSetUp[myCase].dt;
    C.NoOfTimesteps = casesSetUp[myCase].timesteps;
    
    if(AMRlvl > 0) {
        C.AdaptiveMeshRefinement = true;
        C.activeAMRlevelIndicators.Add(
            new AMRonNarrowband() { maxRefinementLevel = AMRlvl }
        );
    }
    
    if(bInitial) {
        C.AddInitialValue("VelocityX#A", analyticVel[myCase].velX);
        //C.AddInitialValue("VelocityX#B", anaVel[myCase.Case - 1, 0]);
        C.AddInitialValue("VelocityY#A", analyticVel[myCase].velY);
        //C.AddInitialValue("VelocityY#B", anaVel[myCase.Case - 1, 1]);
        C.AddInitialValue("VelocityZ#A", analyticVel[myCase].velZ);
        //C.AddInitialValue("VelocityZ#B", anaVel[myCase.Case - 1, 2]);
    }
    
    C.PostprocessingModules.Add(new SphericalHarmonicsLogging() { MaxL = 8, RotSymmetric = true });
    C.PostprocessingModules.Add(new DropletMetricsLogging() { AxisSymmetric = true });
    C.PostprocessingModules.Add(new EnergyLogging());
    
    C.TracingNamespaces = "*";
    
    Controls.Add(C);
    
}
}
}
}
}
}

Case 1: OD3D_J432k3_wallBC_amr1_m2_Oh01_eta04
Case 2: OD3D_J432k3_wallBC_amr1_m2_Oh01_eta02
Case 3: OD3D_J432k3_wallBC_amr1_m2_Oh01_eta01
Case 4: OD3D_J432k3_wallBC_amr1_m3_Oh01_eta04
Case 5: OD3D_J432k3_wallBC_amr1_m4_Oh01_eta04
Case 6: OD3D_J432k3_wallBC_amr1_m4_Oh01_eta01
Case 7: OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005


In [ ]:
int NC = Controls.Count;
for(int i = 0; i < NC; i++) {
    for(int j = 0; j < NC; j++) {
        if(i == j)
            Assert.IsTrue(Controls[i].Equals(Controls[j]), "Control is not self-equal for " + i);
        else
            Assert.IsFalse(Controls[i].Equals(Controls[j]), "Different Control are wrongly equal for " + i + " and " + j);
    }
}

## Launch Jobs

In [ ]:
Controls.Select(C => C.SessionName)

index,value
0,OD3D_J1024k3_refinedTest_amr0_case2_Oh0.1_StartUp


In [ ]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Deploying job OD3D_J1024k3_refinedTest_amr0_case2_Oh0.1_StartUp ... 
Deploying executables and additional files ...
Deployment directory: D:\local\binaries\OscillatingDroplet3D-XNSE_Solver2022Mar25_083116
copied 50 files.
   written file: control.obj
deployment finished.
Mini batch processor is already running.



In [ ]:
wmg.AllJobs

#0: OD3D_J1024k3_refinedTest_amr0_case2_Oh0.1_StartUp: PendingInExecutionQueue (MiniBatchProcessor client  LocalPC @D:\local\binaries)	OD3D_J1024k3_refinedTest_amr0_case2_Oh0.1_StartUp: InProgress (MiniBatchProcessor client  LocalPC @D:\local\binaries)


In [ ]:
// wait for all jobs to finish (up to 5 days, check every 30 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600), PollingIntervallSeconds:(60*30));

Error: System.OperationCanceledException: Command :SubmitCode: // wait for all jobs to finish (up to 5 days, chec ... cancelled.

In [ ]:
// detect failed Jobs in the job management
/*
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects
*/

In [ ]:
//suspects.Count()

In [ ]:
//NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

### Inspect the output of some arbitrary job:

In [ ]:
BoSSSshell.WorkflowMgm.AllJobs.First().Value.ShowOutput();

Starting external console ...
(You may close the new window at any time, the job will continue.)


In [ ]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()